In [5]:
import matplotlib as plt
import requests
from bs4 import BeautifulSoup
import pandas as pd

headers = {"User-Agent": 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/78.0.3904.97 Safari/537.36'}

In [10]:
## Preço máximo
pmax = 550000

## Área mínima
amin = 40

## Bairro
reg = 'copacabana'

## Quantidade de imóveis disponíveis
#int(soup.find('h1', class_='js-summary-title heading-regular heading-regular__bold align-left results__title').text.split(' ')[0])

In [14]:
lista = []

for pg in range(10):
    url = 'https://www.zapimoveis.com.br/venda/imoveis/rj+rio-de-janeiro+zona-sul+'+reg+'/?pagina='+str(pg)+'&onde=,Rio%20de%20Janeiro,Rio%20de%20Janeiro,Zona%20Sul,'+reg.capitalize()+',,BR%3ERio%20de%20Janeiro%3ENULL%3ERio%20de%20Janeiro%3EZona%20Sul%3E'+reg.capitalize()+',-22.951193,-43.180784&transacao=Venda&precoMaximo='+str(pmax)+'&areaMinima='+str(amin)+'&tipo=Im%C3%B3vel%20usado'
    page = requests.get(url, headers = headers)
    soup = BeautifulSoup(page.content, 'html.parser')
    containers = soup.find_all('div', class_='box--display-flex box--flex-column gutter-top-double gutter-left-double gutter-right-double gutter-bottom-double simple-card__box')
    
    for i in containers:
        ## Hash card
        hash_= hash(i)
        ## Get address OK
        address = i.find('p', class_='color-dark text-regular simple-card__address').text
        street = address.split(', ')[0]
        region = address.split(', ',1)[1]
        ## Get condominium OK
        condt = i.find(class_='card-price__item condominium text-regular')
        try:
            cond = int(str(condt).split('R$ ')[1].split('<')[0])
        except:
            cond = None
        ## Get IPTU OK
        iptut = i.find(class_='card-price__item iptu text-regular')
        try:
            iptu = int(str(iptut).split('R$ ')[1].split('<')[0])
        except:
            iptu = None
        ## Get bedrooms OK
        bdrms = i.find('li', class_='feature__item text-small js-bedrooms')
        try:
            bedrooms = int(bdrms.text.strip().split(' ')[0])
        except:
            bedrooms = None
        ## Get size OK
        info = i.find_all('li', 'feature__item text-small')
        size = int(info[0].span.text.strip().split(' ')[0])
        ## Get restrooms OK
        try:
            restrooms = int(info[2].span.text.strip().split(' ')[0]) if len(info) == 3 else int(info[1].span.text.strip())
        except:
            restrooms = None
        ## Get parking OK
        parking = int(info[1].span.text.strip().split(' ')[0]) if len(info) == 3 else 0
        ## Get price OK
        price = int(i.find('p', class_='simple-card__price js-price heading-regular heading-regular__bolder align-left').text.strip().split('R$ ')[1].replace(".",""))
        ## Get comments
        comment = i.find('span', class_='text-regular simple-card__text')
        ## Generate list
        lista.append([street, region, size, bedrooms, restrooms, parking, cond, iptu, price, hash_])

## List to dataframe
a = pd.DataFrame(lista, columns = ['Street', 'Region', 'Size', 'Bedrooms', 'Restrooms', 'Parking', 'Cond', 'IPTU', 'Price', 'Hash'])
a = a.drop_duplicates(inplace=False)
## m² Price
a['Price/m2'] = (a['Price'] / a['Size']).round(0)

## Only selected region
a = a[a['Region'] == reg.capitalize()]

print('cell done')

cell done


In [15]:
## Filters
size_f = a['Size'] > 50
cond_f = a['Cond'] <= 800
rest_f = a['Restrooms'] > 1
price_f = a['Price'] <= 520000

a = a[size_f & cond_f & rest_f & price_f].sort_values(by =['Price']).reset_index(inplace=False, drop=True)

a.reset_index(inplace=False, drop=True)
a.sort_values(by =['Price'], ascending=True)#.reset_index(inplace=False, drop=True)

print('cell done')

cell done


In [16]:
#exc_hash = [729212732937898415, 824045975498986750]
#exc_i = a.index[a['Hash'] == exc_hash

#a.drop(exc_i, inplace=True)

a

,Street,Region,Size,Bedrooms,Restrooms,Parking,Cond,IPTU,Price,Hash,Price/m2
0,Rua Sá Ferreira,Copacabana,55,1.0,3.0,0,594.0,302.0,442000,-7320885696677062696,8036.0
1,Rua Saint Roman,Copacabana,128,3.0,3.0,0,460.0,302.0,450000,-1802334741407362169,3516.0
2,Praça Vereador Rocha Leão,Copacabana,66,2.0,2.0,0,725.0,994.0,470000,-2221806099371811144,7121.0
3,Rua Sá Ferreira,Copacabana,70,2.0,2.0,0,650.0,NaN,485000,927907724486400598,6929.0
4,Rua República do Peru,Copacabana,80,2.0,2.0,1,700.0,NaN,490000,-1132386582055330624,6125.0
5,Rua Siqueira Campos,Copacabana,60,1.0,2.0,2,728.0,970.0,498000,-3320043379695629330,8300.0
6,Avenida Nossa Senhora de Copacabana,Copacabana,52,1.0,2.0,0,410.0,NaN,500000,8357706557611518088,9615.0
7,Rua Ministro Alfredo Valadão,Copacabana,65,2.0,2.0,1,570.0,123.0,500000,-81202180211105776,7692.0
8,Rua Ministro Alfredo Valadão,Copacabana,65,2.0,2.0,0,600.0,800.0,500000,8528447634679370933,7692.0
9,Rua Siqueira Campos,Copacabana,90,2.0,2.0,0,700.0,760.0,520000,-8320199805992369068,5778.0
